# Introduction

this notebook demos example of using llm in a MPS backend (apple silicon GPU) using torch 2.x

Referece:
* torch 2.x MPS Backend: https://pytorch.org/docs/stable/notes/mps.html

In [1]:
import os
import torch
import applyllm as apl

print(apl.__version__)


0.0.6


In [2]:
# check that MPS is availabe (Metal Performance Shaders)
if not torch.backends.mps.is_available():
    print("MPS is not available")
else:
    print("MPS is available")
    mps_device = torch.device("mps")
    print(mps_device)



MPS is available
mps


## Define global variables

In [3]:
from applyllm.accelerators import (
    DirectorySetting,
    TokenHelper as th,
)
    
dir_mode_map = {
    "kf_notebook": DirectorySetting(),
    "mac_local": DirectorySetting(home_dir="/Users/yingding", transformers_cache_home="MODELS", huggingface_token_file="MODELS/.huggingface_token"),
}

model_map = {
    "llama7B-chat":     "meta-llama/Llama-2-7b-chat-hf",
    "llama13B-chat" :   "meta-llama/Llama-2-13b-chat-hf",
    "llama70B-chat" :   "meta-llama/Llama-2-70b-chat-hf",
    "mistral7B-01":     "mistralai/Mistral-7B-v0.1",
    "mistral7B-inst02": "mistralai/Mistral-7B-Instruct-v0.2",
    "mixtral8x7B-01":   "mistralai/Mixtral-8x7B-v0.1",
    "mixtral8x7B-inst01":   "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "gemma7b-it": "google/gemma-7b-it",
    "gemma7b" : "google/gemma-7b",
    "gemma2b-it": "google/gemma-2b-it",
    "gemma2b" : "google/gemma-2b",
}

default_model_type = "mistral7B-01"
default_dir_mode = "mac_local"

dir_setting = dir_mode_map[default_dir_mode]

os.environ["WORLD_SIZE"] = "1" 
os.environ['XDG_CACHE_HOME'] = dir_setting.get_cache_home()

print(os.environ['XDG_CACHE_HOME'])

/Users/yingding/MODELS


In [4]:
import transformers
import torch

print(transformers.__version__)
print(torch.__version__)

4.39.3
2.2.2


## Choose LLM model

In [5]:
# model_type = default_model_type
# model_type = "gemma7b-it"
model_type = "gemma2b-it"
# model_type = "mistral7B-inst02"
# model_type = "llama7B-chat"
# model_type = "llama13B-chat"

model_name = model_map.get(model_type, default_model_type)
print(model_name)

google/gemma-2b-it


### Fast tokenizer

* https://github.com/huggingface/transformers/issues/23889#issuecomment-1584090357

### Load LLM Model and then Tokenizer

In [6]:
from applyllm.pipelines import (
    ModelCatalog,
    KwargsBuilder
)
token_kwargs = th.gen_token_kwargs(model_type=model_type, dir_setting=dir_setting)
print(f"token_kwargs: {token_kwargs}")

# data_type = torch.bfloat16
data_type = torch.float16
device_map = "auto" # "mps" # "auto"  
# auto caste not working for mps 4.38.2
# https://github.com/huggingface/transformers/issues/29431 

# mixtral model has no max_new_tokens limit, so it is not set here.
model_kwargs = {
    "torch_dtype": data_type, #bfloat16 is not supported on MPS backend, float16 only on GPU accelerator
    # torch_dtype=torch.float32,
    # max_length=MAX_LENGTH,
    "device_map": device_map,
    "max_length" : None, # remove the total length of the generated response
}
print(f"model_kwargs: {model_kwargs}")

# set the transformers.pipeline kwargs
# the torch_dtype shall be set both for the model and the pipeline, due to a transformer issue.
# otherwise it will cause unnecessary more memory usage in the pipeline of transformers
# https://github.com/huggingface/transformers/issues/28817
# https://github.com/mlflow/mlflow/pull/10979

# Set transformers.pipeline only to return generated text return_full_text=False
# https://github.com/huggingface/transformers/issues/17117#issuecomment-1120809167
pipeline_kwargs = {
    "task": "text-generation",
    "max_new_tokens" : 200,
    "do_sample" : True, # do_sample True is required for temperature
    "temperature" : 0.001, 
    "device_map" : device_map, # use the MPS device if available
    "top_k": 3,
    "top_p": 0.95,
    # "num_return_sequences": 1,
    "framework": "pt", # use pytorch as framework
    "return_full_text": False, # return only the generated text, not the input text with the generated text
}

gemma_pipeline_kwargs = {
    "add_special_tokens": True,
    "torch_dtype": data_type,
}

# pipeline_kwargs override the model_kwargs during the merge
pipeline_kwargs = KwargsBuilder([model_kwargs]).override(pipeline_kwargs).build()

if model_name.startswith(ModelCatalog.GOOGLE_FAMILY):
    pipeline_kwargs = KwargsBuilder([pipeline_kwargs]).override(gemma_pipeline_kwargs).build()

print(f"pipeline_kwargs: {pipeline_kwargs}")


huggingface token is NOT needed
token_kwargs: {}
model_kwargs: {'torch_dtype': torch.float16, 'device_map': 'auto', 'max_length': None}
pipeline_kwargs: {'torch_dtype': torch.float16, 'device_map': 'auto', 'max_length': None, 'task': 'text-generation', 'max_new_tokens': 200, 'do_sample': True, 'temperature': 0.001, 'top_k': 3, 'top_p': 0.95, 'framework': 'pt', 'return_full_text': False, 'add_special_tokens': True}


### Max memory to offload parts of LLM model to the CPU memory
* https://huggingface.co/docs/accelerate/concept_guides/big_model_inference#designing-a-device-map

Note:
* Max Memory offload to CPU is CUDA implementation only



In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from applyllm.utils import time_func
from applyllm.pipelines import ModelConfig, LocalCausalLMConfig


base_lm_config = ModelConfig(
  model_config = {
    "pretrained_model_name_or_path": model_name,
    "device_map": device_map,
  }
)

# No bitsandbytes qunatization support for MPS backend yet, set quantized to False
kwargs = {
  "quantized": True, # False,
  "model_config": base_lm_config.get_config(),
  "quantization_config": {
    "quantization_config": transformers.BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16
      )
    },
  "load_in_8bit": False # MPS only
}

lm_config = LocalCausalLMConfig(**kwargs)

@time_func
def load_model():
  return AutoModelForCausalLM.from_pretrained(    
    **lm_config.get_config(),
    **token_kwargs,  
  )

model = load_model()

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
tokenizer_kwargs = {
    "model_config": {
        "pretrained_model_name_or_path": model_name,
        "device": "cpu",
        # "device_map": "auto", # put to GPU if GPU is available
        # "max_position_embeddings": MAX_LENGTH,
        # "max_length": MAX_LENGTH,
    },
}
tokenizer_config = ModelConfig(**tokenizer_kwargs)

tokenizer = AutoTokenizer.from_pretrained(
    **tokenizer_config.get_config(),
    **token_kwargs
)

In [ ]:
tokenizer

### Testing token
* https://huggingface.co/docs/tokenizers/pipeline

In [ ]:
print(model_name)

In [ ]:
from applyllm.pipelines import (
    ModelCatalog,
    PromptHelper
)

model_info = ModelCatalog.get_model_info(model_name)
prompt_helper = PromptHelper(model_info)

if model_info.model_family == ModelCatalog.GOOGLE_FAMILY:
    query = """BEGIN EXAMPLE
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
END EXAMPLE

Your turn:            
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? 
"""
    inputs=[prompt_helper.gen_prompt(query)]
else: 
    inputs=["""
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""]

In [ ]:
input_test_encoded = tokenizer.encode(inputs[0])
print(f"{len(input_test_encoded)}")
print(input_test_encoded)

In [ ]:
response_test_decoded = tokenizer.decode(input_test_encoded)
print(response_test_decoded)

### Load LLM

In [ ]:
# bitsandbytes quantization does not work with MPS backend
print(pipeline_kwargs)

# transformer pipeline kwargs
tp_kwargs = {
    "model": model,
    "tokenizer": tokenizer,
}

tp_config = ModelConfig(model_config = tp_kwargs)

generator = transformers.pipeline(
    **tp_config.get_config(),
    **pipeline_kwargs,
    **token_kwargs,
    # **compression_kwargs,
)

##### Install autopep8 or black extension in VSCode
`shift + opt + F` to auto format python code

In [ ]:
from applyllm.accelerators import AcceleratorStatus

gpu_status = AcceleratorStatus.create_accelerator_status()
gpu_status.gpu_usage()

In [ ]:
import pydantic
pydantic.__version__

In [ ]:
from pprint import pprint
from langchain import PromptTemplate, LLMChain
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline


llm = HuggingFacePipeline(
    pipeline=generator 
)

template = prompt_helper.gen_prompt("{input}")
prompt = PromptTemplate(template=template, input_variables=["input"])


@time_func
def chat(input) -> str:
    """
    Args: 
        input: str - the input text to chat with the model, e.g. inputs[0]
    """
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    # print(repr(llm_chain))
    dict_response = llm_chain.invoke(input={"input": input})
    return dict_response.get("text", "")

# pprint(response, indent=0, width=100)

# response = chat(input=inputs[0])
# print(response)

In [ ]:
repeat = 1
for i in range(repeat):
    response = chat(input=inputs[0])
    print(response)

In [ ]:
import gc
def clear_mps_memory(tokenizer, generator):
    """clear the MPS memory"""
    if tokenizer is not None:
        del tokenizer
    if generator is not None:
        # need to move the model to cpu before delete.
        generator.model.cpu()
        del generator
    gc.collect()
    torch.mps.empty_cache()
    # report the GPU usage
    gpu_status.gpu_usage()


In [ ]:
gpu_status.gpu_usage()

In [ ]:
# inputs2 = ["Which animal is the largest mammal?"]
# inputs2 = ["Can you tell me something about chron's disease?"]

# hallucination https://www.findacode.com/snomed/34000006--crohns-disease.html

# real answer is 34000006, probably need a RAG 
# inputs2 = ["Which snomed ct code has chron's disease?"]

# inputs2 = ["Can you tell me more about the company nordcloud?"]
inputs2 = ["Can you tell me more about the company nordcloud in munich?"]

In [ ]:
print(chat(input=inputs2[0]))

## Define agent
* https://python.langchain.com/docs/modules/agents/
* https://python.langchain.com/docs/modules/agents/quick_start/
* https://python.langchain.com/docs/modules/agents/how_to/custom_agent/

## Gemma with custom langchain tool 
* https://github.com/Ashufet/LangChain_ReAct-Agent-with-Function-Calling_Ollama-Gemma-LLM_LangSmith
* ReAct Agent: https://www.youtube.com/watch?v=exYUJcz4uZs

## Ollama local host endpoint with LangChain
* https://medium.com/the-constellar-digital-technology-blog/geek-out-time-play-with-langchain-2-locally-with-gemma-96c6ca370649

## ReAct Agent
* https://python.langchain.com/docs/modules/agents/agent_types/react/

## Retrieval Agent examples
* https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/08-langchain-retrieval-agent.ipynb

In [ ]:
embed_model_map = {
    "sentence-transformers": "sentence-transformers/all-MiniLM-L12-v2", # 384
    "baai" : "BAAI/bge-base-en-v1.5" # 768 embedding dims
}

# embed_model_vendor = "sentence-transformers"
embed_model_vendor = "baai"

In [ ]:
embed_model_name = embed_model_map[embed_model_vendor]

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_config = {
    "model_name" : embed_model_name,
    "model_kwargs": {'device': 'cpu'},
    "encode_kwargs": {'normalize_embeddings': True}
}
embed_config = ModelConfig(model_config=model_config)

# is downloaded at "{MODEL_CACHE_DIR}/models/torch/sentence_transformer" folder
embed_model = HuggingFaceEmbeddings(
    **embed_config.get_config()
)

embed_config

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from applyllm.utils import token_size
from applyllm.pipelines import ModelConfig

MAX_POSITION_EMBEDDINGS = 1000 # 4096
CHUNK_SIZE = (MAX_POSITION_EMBEDDINGS // 1000) * 1000

# Config splitter
model_config = {
    # Set a really small chunk size, just to show.
    "chunk_size": CHUNK_SIZE,
    "chunk_overlap": 200,
    "length_function": token_size, # len,
    "is_separator_regex": False,
}

splitter_config = ModelConfig(model_config=model_config)

loader = WebBaseLoader("https://nordcloud.com/company/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    **splitter_config.get_config()
).split_documents(docs)

vector = FAISS.from_documents(documents, embed_model)
retriever = vector.as_retriever()

In [ ]:
retriever.get_relevant_documents(inputs2[0])[0]

In [ ]:
from langchain.chains import RetrievalQA

# chain_type = "map_reduce"
# chain_type = "stuff"
chain_type = "refine" 
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    # combine_docs_chain_kwargs={'prompt': reduce_prompt_template},
    # chain_type_kwargs={"map_prompt": map_prompt_template},
    return_source_documents=True,
    verbose=True,
    )

In [ ]:
import langchain

query = inputs2[0]
# langchain.debug = True
qa.invoke({"query": query})
# langchain.debug = False

# Tools
* https://github.com/langchain-ai/langchain/issues/14954#issuecomment-1864918697
* https://github.com/langchain-ai/langchain/issues/14954#issuecomment-1876906769

In [ ]:
from langchain.agents import Tool

# tools = [
#     Tool(
#         name='Knowledge Base',
#         func=qa.invoke,
#         # func=qa.invoke,
#         description=(
#             'use this tool when answering general knowledge queries to get '
#             'more information about the topic'
#         )
#     )
# ]

tools = [
    Tool(
        name='northcloud_search',
        func=qa.invoke,
        # func=qa.invoke,
        description=(
            'Search for information about NorthCloud. '
            'For any questions about NorthCloud, you must use this tool!'
        )
    )
]

In [ ]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType 

agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    output_key = "result",
    handle_parsing_errors=True,
    early_stopping_method='generate',
    memory = ConversationBufferMemory(memory_key = 'chat_history')   
)

In [ ]:
agent.invoke({"input": inputs2[0]})

In [ ]:
# from langchain.agents import AgentExecutor
# https://github.com/langchain-ai/langchain/issues/14954#issuecomment-1864918697
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=False, handle_parsing_errors=True)

In [ ]:
# agent_executor.invoke({"input": inputs2[0]})
# agent_executor.invoke({"input": "hi"})

In [ ]:
CLEAR_MEMORY = False
# CLEAR_MEMORY = True

if CLEAR_MEMORY:
    clear_mps_memory(tokenizer=tokenizer, generator=generator)